In [ ]:
#
# Universidad EAFIT 
# 2025-2
# SI7016 - NLP Aplicado - Lecture 08
#

# Ejemplo 1: Creación de un Agente LLM en Python

In [ ]:
# instalar dependencias
%pip install -U langchain langchain-openai langchain-community wikipedia openai faiss-cpu tiktoken    


In [ ]:
# api key from env

import os

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
HF_TOKEN = os.environ.get("HF_TOKEN")
LANGCHAIN_API_KEY= os.environ["LANGCHAIN_API_KEY"]
SERPAPI_API_KEY = os.environ["SERPAPI_API_KEY"]

from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.hub import pull
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.tools import tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

llm = ChatOpenAI(model="gpt-4", temperature=0.5)

# --- Herramienta: Wikipedia (en español) ---
wiki = WikipediaAPIWrapper(lang="es", top_k_results=3, doc_content_chars_max=4000)

@tool("wikipedia")
def search_wikipedia(query: str) -> str:
    """Busca información en Wikipedia (ES). Devuelve un resumen breve de los resultados."""
    return wiki.run(query)

tools = [search_wikipedia]

# --- Prompt ReAct (usamos un prompt público del hub) ---
# "react-chat" está adaptado a chat + herramientas; si prefieres sin chat, usa "react".
prompt = pull("hwchase17/react-chat")

# --- Crear agente ReAct + ejecutor ---
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# --- Memoria de conversación (RunnableWithMessageHistory) ---
# En vez de ConversationBufferMemory, el patrón recomendado es envolver el runnable
# con un store de historiales por sesión.
_store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in _store:
        _store[session_id] = ChatMessageHistory()
    return _store[session_id]

# El AgentExecutor espera la clave de entrada "input" y puede devolver "output".
chain_with_history = RunnableWithMessageHistory(
    executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="output",
)

# --- Ejemplo de interacción ---
session_id = "demo-profesor"  # cámbialo por el id de usuario/sesión que quieras persistir

resp1 = chain_with_history.invoke(
    {"input": "¿Quién fue Alan Turing? Por favor, resume en 3 líneas y cita la fuente si usas Wikipedia."},
    config={"configurable": {"session_id": session_id}},
)
print(resp1["output"])

# La memoria guarda el contexto para la siguiente pregunta:
resp2 = chain_with_history.invoke(
    {"input": "Amplía con 2 aportes clave adicionales y una fecha importante."},
    config={"configurable": {"session_id": session_id}},
)
print(resp2["output"])


# Ejemplo 2: Agente con Razonamiento y Planificación

In [ ]:
%pip install google-search-results

# Este agente puede recibir objetivos y descomponerlos en tareas usando un marco de planificación.

# se requiere la API KEY de SerpAPI - https://serpapi.

# realice el registro en línea, tiene 100 busquedas gratuitas al men, uso no comercial

# crera la variable con el valor SERPAPI_API_KEY

# en el bash shell:

# bash$ set SERPAPI_API_KEY=123...232
# bash$ export SERPAPI_API_KEY
# bash$ echo $SERPAPI_API_KEY
# 123...232

#import os
#os.environ["SERPAPI_API_KEY"] = "TU_CLAVE_API"


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.messages import SystemMessage  # en vez de langchain.schema

# LLM
llm = ChatOpenAI(model=os.getenv("MODEL_NAME", "gpt-4o-mini"), temperature=0.2)

# Herramientas
tools = load_tools(["serpapi"], serpapi_api_key=os.environ["SERPAPI_API_KEY"])

# (Opcional) mensaje de sistema — NO es usado por ZERO_SHOT_REACT_DESCRIPTION.
# Si quieres aplicar un prefijo personalizado, mejor usa agent_kwargs={"prefix": "..."}.
system_message = SystemMessage(
    content="Eres un agente de IA que descompone problemas complejos en pasos más pequeños y ejecuta acciones."
)

# Agente ReAct (zero-shot)
agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    # agent_kwargs={"prefix": system_message.content},  # <- usa esto si quieres inyectar tu “system”
)

# Ejecutar
result = agent_executor.run("Encuentra las últimas noticias sobre inteligencia artificial.")
print(result)


In [ ]:
%pip install openai-agents

In [ ]:
from agents import Agent, Runner

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

# Intended for Jupyter notebooks where there's an existing event loop
result = await Runner.run(agent, "Write a haiku about recursion in programming.")  # type: ignore[top-level-await]  # noqa: F704
print(result.final_output)

# Code within code loops,
# Infinite mirrors reflect—
# Logic folds on self.

In [ ]:
# funciona como un programa aparte: ejemplo.py
from agents import Agent, Runner

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

# Intended for Jupyter notebooks where there's an existing event loop
result = await Runner.run(agent, "Write a haiku about recursion in programming.")  # type: ignore[top-level-await]  # noqa: F704
print(result.final_output)

# Code within code loops,
# Infinite mirrors reflect—
# Logic folds on self.

In [ ]:
import nest_asyncio
import asyncio
from agents import Agent, Runner

print(dir(Runner))  # Verifica qué métodos están disponibles en Runner


In [ ]:
# ejecución asincrónica dentro de notebook:

import nest_asyncio
#import asyncio
from agents import Agent, Runner

nest_asyncio.apply()  # Permite usar asyncio en Jupyter sin conflictos

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

async def main():
    result = await Runner.run(agent, "Write a haiku about recursion in programming.")
    print(result.final_output)

await main()  # Ejecutar directamente en Jupyter


In [ ]:
import nest_asyncio
nest_asyncio.apply()


In [ ]:
import asyncio
from agents import Agent, Runner

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

async def main():
    result = await Runner.run(agent, "Write a haiku about recursion in programming.")
    print(result.final_output)

asyncio.run(main())


# Ejemplo 3 - instalación basica de llamaindex

In [ ]:
%pip install llama-index-llms-huggingface-api llama-index-embeddings-huggingface


In [ ]:
# Opción 1: Usar el modelo como chat (recomendado)

import os
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.llms import ChatMessage, MessageRole

HF_TOKEN = os.environ["HF_TOKEN"]  # asegúrate de exportarlo

llm = HuggingFaceInferenceAPI(
    model_name="Qwen/Qwen2.5-Coder-32B-Instruct",
    temperature=0.7,
    max_tokens=100,   # alias a max_new_tokens en la API
    token=HF_TOKEN,
)

# Usa .chat con una lista de mensajes
msgs = [
    ChatMessage(role=MessageRole.SYSTEM, content="You are a helpful coding assistant."),
    ChatMessage(role=MessageRole.USER, content="Hello, how are you?")
]
resp = llm.chat(msgs)

print(resp.message.content)
